# NFL Baseline

# import libraries

In [ ]:
# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import math

import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
import timm


# deep learning
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import StratifiedKFold

# loss metrics
from sklearn.metrics import matthews_corrcoef

# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")

# Set Configurations

In [ ]:
kaggle = False
DEBUG = True
class CFG:
    if kaggle:
        BASE_DIR = "/kaggle/input/nfl-player-contact-detection"
    else:
        BASE_DIR = "/workspace/input"
    TRAIN_HELMET_CSV = os.path.join(BASE_DIR, "train_baseline_helmets.csv")
    TRAIN_TRACKING_CSV = os.path.join(BASE_DIR, "train_player_tracking.csv")
    TRAIN_VIDEO_META_CSV = os.path.join(BASE_DIR, "train_video_metadata.csv")
    TRAIN_LABEL_CSV = os.path.join(BASE_DIR, "train_labels.csv")
    TARGET_CSV = os.path.join("/kaggle/input/dfl-creatatraindataset-helmet/target_fillna0.csv")
    TRAIN_IMG_DIR = "/kaggle/input/nfl-baseline-saveframes"
    # data config    
    img_size = (224, 224)
    batch_size = 64
    num_workers = 0
    n_fold = 1
    masksize_helmet_ratio = 4 # helmetサイズにこの係数をかけたサイズだけ色を残して後は黒塗りする

    # model config
    model_name = "tf_efficientnet_b0"
    out_features = 1
    inp_channels= 3
    pretrained = True
    model_dir = os.path.join(os.path.dirname(BASE_DIR), "model")
    if kaggle:
        model_dir = "/kaggle/working"
    
    # learning config
    n_fold = 5
    train_fold = [0, 1, 2, 3, 4]
    n_epoch = 20
    lr = 1e-6
    T_max = 10
    min_lr = 1e-7
    weight_decay = 1e-6
    
    # etc
    print_freq = 100
    random_seed = 21
    
    MLFLOW_CATEGORY = "make_baseline"
    EXP_NAME = "DEBUG"
    if DEBUG:
        n_epoch = 3
        batch_size=4
        train_fold = [0, 1]
        # epoch_step_valid = 0
        # steps_per_epoch = 10
        

# Utils

In [ ]:
def seed_everything(seed=CFG.random_seed):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False
seed_everything()

# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

In [ ]:
def asMinutes(s):
    """Convert Seconds to Minutes."""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    """Accessing and Converting Time Data."""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Dataset Utils

In [ ]:
def set_inimg_window(crop_pos, mask_size, img_size=(720, 1280)):#crop_pos = [left, top, right, bot]
    if mask_size[1] >= img_size[0]:
        top, bot = 0, img_size[1]
    else:
        top=(crop_pos[1] + crop_pos[3])//2 - mask_size[1]//2
        bot=(crop_pos[1] + crop_pos[3])//2 + mask_size[1]//2
        if top < 0:
            bot = bot - top
            top = 0
        elif bot > img_size[0]:
            top = top - (bot-img_size[0])
            bot = img_size[0]

    if mask_size[0] >= img_size[1]:
        left, right = 0, img_size[1]
    else:
        left = (crop_pos[0] + crop_pos[2])//2 - mask_size[0]//2
        right = (crop_pos[0] + crop_pos[2])//2 + mask_size[0]//2
        if left < 0:
            right = right - left
            left = 0
        elif right > img_size[1]:
            left = left - (right - img_size[1])
            right = img_size[1]
    crop_area = np.array([left, top, right, bot]).astype(np.int)
    return crop_area

In [ ]:
def get_crop_area(p1_helmet, p2_helmet, input_size=(720, 1280), output_size=(448, 448)):
    p1_x_center, p1_y_center = p1_helmet[0] + p1_helmet[2]//2, p1_helmet[1] + p1_helmet[3]//2
    p2_x_center, p2_y_center = p2_helmet[0] + p2_helmet[2]//2, p2_helmet[1] + p2_helmet[3]//2
    if p1_helmet[2] > 0 and p2_helmet[2] > 0:
        crop_x_center, crop_y_center = (p1_x_center + p2_x_center)//2, (p1_y_center + p2_y_center)//2
    elif p1_helmet[2] > 0:
        crop_x_center, crop_y_center = p1_x_center, p1_y_center
    elif p2_helmet[2] > 0:
        crop_x_center, crop_y_center = p2_x_center, p2_y_center
    else:
        crop_area = [0, 0, input_size[1], input_size[0]]
#         crop_x_center, crop_y_center = p2_x_center, p2_y_center
        return crop_area
    crop_left = crop_x_center - output_size[1]//2
    crop_top = crop_y_center - output_size[0]//2
    crop_right = crop_x_center + output_size[1]//2
    crop_bot = crop_y_center + output_size[0]//2
    crop_area = [crop_left, crop_top, crop_right, crop_bot]
    mask_size = (crop_right-crop_left, crop_right-crop_left)
    crop_area = set_inimg_window(crop_area, mask_size)
    return crop_area

In [ ]:
def get_playermasked_img(img, helmet_pos, img_size=(720, 1280, 3)):#helmet pos = [left, width, top, height]
    if helmet_pos[2] == 0:
        return img
    mask_size=(helmet_pos[1]+helmet_pos[3]/2)*CFG.masksize_helmet_ratio# helmetの大きさによってplayerの範囲も変更
    helmet_area = [helmet_pos[0], helmet_pos[2], helmet_pos[0]+helmet_pos[1], helmet_pos[2]+helmet_pos[3]]#[left, top, right, bot]
    player_area = set_inimg_window(helmet_area, (mask_size,mask_size))
    mask = np.zeros(img_size, dtype=np.float)
    cv2.rectangle(mask, [player_area[0], player_area[1]], [player_area[2], player_area[3]], (255, 255, 255), -1)
    mask = np.clip(mask, 0, 1).astype(np.float)
    return mask

# Load Data

In [ ]:
target_df = pd.read_csv(CFG.TARGET_CSV)
# target_game_plays = target_df["game_play"].unique()[:50]
target_game_plays = target_df["game_play"].unique()[:5]
target_df = target_df[target_df["game_play"].isin(target_game_plays)]
print(len(target_df))
if DEBUG:
    target_df = target_df.sample(20000)

In [ ]:
display(target_df.head())
print(target_df.columns)

In [ ]:
contact_df = target_df.query('contact==1')
contact_df["E_width_1"].hist(bins=100)
print(len(contact_df.query('E_width_1==0')), len(contact_df), len(contact_df.query('E_width_1==0'))/len(contact_df)*100)

Endzoneのviewのplayer1では全体の14%ぐらいがヘルメット検知なしになっている。

# Dataset

In [ ]:
class NFLDataset(Dataset):
    def __init__(self, target_df, transform=None):
        self.target_df = target_df
        self.transform = transform

    def __len__(self):
        return len(self.target_df)

    def __getitem__(self, idx):
        target_info = self.target_df.iloc[idx]
        target = target_info.contact
        # read frame image
        game_play = target_info.game_play
        frame = target_info.frame
#         file_id = f"{game_play}_{view}_{frame:05}.png"
        file_id = f"{game_play}_Endzone_{frame:05}.png"
        filename = os.path.join(CFG.TRAIN_IMG_DIR, file_id)
        img = cv2.imread(filename)
        if img is None:
            img = np.zeros((224, 224, 3))
            img = np.transpose(img, (2, 0, 1)).astype(np.float)
            img = torch.tensor(img, dtype=torch.float)
            return img, target
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # player highlight mask
        player1 = target_info.nfl_player_id_1
        player2 = target_info.nfl_player_id_2
        p1_helmet = np.array([target_info.E_left_1, target_info.E_width_1,
                            target_info.E_top_1, target_info.E_height_1]).astype(np.int)
        p2_helmet = np.array([target_info.E_left_2, target_info.E_width_2,
                            target_info.E_top_2, target_info.E_height_2]).astype(np.int)
        mask1 = get_playermasked_img(img, p1_helmet)
        mask2 = get_playermasked_img(img, p2_helmet)
        mask = np.clip(mask1 + mask2, 0, 1).astype(np.float)
        img = mask*img
        # crop players area
        crop_area = get_crop_area(p1_helmet, p2_helmet)# crop_area=[left, top, right, bot]
#         print(p1_helmet)
#         print(p2_helmet)
#         print(crop_area)
        img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]
        img = cv2.resize(img, dsize=CFG.img_size)
        img = img / 255. # convert to 0-1
        img = np.transpose(img, (2, 0, 1)).astype(np.float)
        img = torch.tensor(img, dtype=torch.float)
        target = torch.tensor(target, dtype=torch.float)
        return img, target

In [ ]:
train_dataset = NFLDataset(target_df)

train_loader = DataLoader(
    train_dataset,
    batch_size = CFG.batch_size,
    shuffle = True,
    num_workers = CFG.num_workers,
    pin_memory = True
)

In [ ]:
show_img_num = 4
for batch_idx, (images, targets) in enumerate(train_loader):
    fig = plt.figure(figsize=(12, 25))
    for idx in range(show_img_num):
        img = images[idx].numpy()
        img = img.transpose((1,2,0))
#         if np.sum(img)>0:
        fig.add_subplot(1,show_img_num ,idx+1)
        plt.imshow(img)
        plt.title(targets[idx].numpy())
    plt.show()
    break

# Model

In [ ]:
# without meta
class NFLNet(nn.Module):
    def __init__(
        self,
        model_name = CFG.model_name,
        out_features = CFG.out_features,
        inp_channels= CFG.inp_channels,
        pretrained = CFG.pretrained
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)
    
    def forward(self, image):
        output = self.model(image)
        return output

# train fn

In [ ]:
def train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler):
    model.train()
    batch_time = AverageMeter()
    losses = AverageMeter()
    start = end = time.time()
    for batch_idx, (images, targets) in enumerate(train_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)                
        preds = model(images)
        
        loss = criterion(preds, targets)
        losses.update(loss.item(), CFG.batch_size) 
        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        loss.backward() # パラメータの勾配を計算
        optimizer.step() # モデル更新
        optimizer.zero_grad() # 勾配の初期化
                
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx % CFG.print_freq == 0 or batch_idx == (len(train_loader)-1):
            print('\t Epoch: [{0}][{1}/{2}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                        epoch, batch_idx, len(train_loader), batch_time=batch_time, loss=losses,
                        remain=timeSince(start, float(batch_idx+1)/len(train_loader)),
            ))
        del preds, images, targets
    gc.collect()
    torch.cuda.empty_cache()
    return losses.avg

# valid fn

In [ ]:
def valid_fn(model, valid_loader, criterion):
    model.eval()# モデルを検証モードに設定
    test_targets = []
    test_preds = []

    batch_time = AverageMeter()
    losses = AverageMeter()
    start = end = time.time()
    for batch_idx, (images, targets) in enumerate(valid_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)
        with torch.no_grad():
            preds = model(images)
            loss = criterion(preds, targets)
        losses.update(loss.item(), CFG.batch_size)
        batch_time.update(time.time() - end)

        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        test_preds.extend(preds)
        test_targets.extend(targets)
        # score = matthews_corrcoef(preds, targets)
        if batch_idx % CFG.print_freq == 0 or batch_idx == (len(valid_loader)-1):
            print('\t EVAL: [{0}/{1}] '
                'Elapsed {remain:s} '
                'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                .format(
                    batch_idx, len(valid_loader), batch_time=batch_time, loss=losses,
                    remain=timeSince(start, float(batch_idx+1)/len(valid_loader)),
                ))
        del preds, images, targets
        gc.collect()
        torch.cuda.empty_cache()
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    return test_targets, test_preds, losses.avg

# Train loop

In [ ]:
def training_loop(target_df):
    
    # set model & learning fn
    model = NFLNet()
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)

    oof_df = pd.DataFrame()
    skf = StratifiedKFold(n_splits = CFG.n_fold, shuffle=True, random_state=CFG.random_seed)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(target_df,target_df["contact"].values)):
        print(f'fold {fold} training start.')        
        # separate train/valid data 
        train_df = target_df.iloc[train_idx]
        valid_df = target_df.iloc[valid_idx]
        train_dataset = NFLDataset(train_df)
        valid_dataset = NFLDataset(valid_df)
        train_loader = DataLoader(train_dataset,batch_size=CFG.batch_size, shuffle = True,
                                    num_workers = CFG.num_workers, pin_memory = True)
        valid_loader = DataLoader(valid_dataset,batch_size=CFG.batch_size, shuffle = True,
                                    num_workers = CFG.num_workers, pin_memory = True)

        # training
        best_score = -np.inf
        start_time = end = time.time()
        for epoch in range(1, CFG.n_epoch + 1):
            print(f'=== epoch: {epoch}: training ===')
            train_loss_avg = train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler)
            valid_targets, valid_preds, valid_loss_avg = valid_fn(model, valid_loader, criterion)

            valid_score = 0
            valid_threshold = 0
            for idx in range(1, 10, 1):
                thr = idx*0.1
                valid_targets = (np.array(valid_targets) > thr).astype(np.int32)
                valid_preds = (np.array(valid_preds) > thr).astype(np.int32)
                score_tmp = matthews_corrcoef(valid_targets, valid_preds)
                if score_tmp > valid_score:
                    valid_score = score_tmp 
                    valid_threshold = thr           
            elapsed = time.time() - start_time
            print(f'epoch:{epoch}, avg train loss:{train_loss_avg:.4f}, avg valid loss:{valid_loss_avg:.4f}, score:{valid_score:.4f}(th={valid_threshold}) ::: time:{elapsed:.2f}s')
            scheduler.step()
            # validationスコアがbestを更新したらモデルを保存する
            if valid_score > best_score:
                best_score = valid_score
                model_name = CFG.model_name
                # torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}_fold{i_fold}.pth')
                torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}.pth')
                print(f'Epoch {epoch} - Save Best Score: {best_score:.4f}. Model is saved.')
                contact_id = valid_df["contact_id"].values
                _oof_df = pd.DataFrame({
                    "contact_id" : contact_id,
                    "pred" : valid_preds,
                    "target" : valid_targets,
                })

        del train_loader, train_dataset, valid_loader, valid_dataset
        gc.collect()
        torch.cuda.empty_cache()
        oof_df = pd.concat([oof_df, _oof_df], axis = 1)
    return oof_df

In [ ]:
oof_df = training_loop(target_df)

In [ ]:
display(oof_df)